### Imports

In [183]:
import pandas as pd
pd.set_option('display.max_columns', None)
import pytz
import duckdb
import time

import requests as rq
import json
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import os
from dotenv import load_dotenv
from pathlib import Path

import asyncio
import json
import websockets
from urllib.parse import urlparse, parse_qs

### Keys

In [65]:
# Loads variables from .env file into environment
load_dotenv()

CG_DEMO_API_KEY = os.getenv("CG_DEMO_API_KEY")
if not CG_DEMO_API_KEY:
    raise RuntimeError("Missing Demo API key in the environment")

CG_PRO_API_KEY = os.getenv("CG_PRO_API_KEY")
if not CG_PRO_API_KEY:
    raise RuntimeError("Missing Pro API key in the environment")

CG_ANALYST_API_KEY = os.getenv("CG_ANALYST_API_KEY")
if not CG_ANALYST_API_KEY:
    raise RuntimeError("Missing Analyst API key in the environment")

### API status

In [7]:
PUB_URL = "https://api.coingecko.com/api/v3"
PRO_URL = "https://pro-api.coingecko.com/api/v3"

In [8]:
def get_response(endpoint, headers, params, URL):

    url = "".join((URL, endpoint))
    response = rq.get(url, headers=headers, params=params)

    try:
        data = response.json()
    except ValueError:
        print("Invalid JSON response")
        return None

    if response.status_code != 200:
        print(f"Failed to fetch data, status code {response.status_code}")

    return data

In [10]:
use_demo = {
           "accept": "application/json",
           "x-cg-demo-api-key" : CG_DEMO_API_KEY
}

use_pro = {
         "accept": "application/json",
         "x-cg-pro-api-key" : CG_PRO_API_KEY
}

In [11]:
get_response("/ping", use_demo, "", PUB_URL)

{'gecko_says': '(V3) To the Moon!'}

### Get new pools

In [14]:
def safe_get(d, path, default=None):
    """Safely get a nested dictionary value."""
    for key in path:
        if isinstance(d, dict) and key in d:
            d = d[key]
        else:
            return default
    return d

def collect_response(list_response):

    response_all = []

    for response in list_response.get("data", []):
        
        all_attributes = response.get("attributes", {})
        rel = response.get("relationships", {})
        
        base_token_add = safe_get(rel, ["base_token", "data", "id"], "NA")
        
        # If token_add exists, split it.
        token_add = base_token_add.split("_")[1] if base_token_add != "NA" and "_" in base_token_add else "NA"
        
        temp_dict = dict(
            pair = safe_get(all_attributes, ["name"], "NA"),
            pool_created_at = safe_get(all_attributes, ["pool_created_at"], "NA"),
            dex = safe_get(rel, ["dex", "data", "id"], "NA"),
            network = safe_get(rel, ["network", "data", "id"], "NA"),
            token_add = token_add,
            pool_add = safe_get(all_attributes, ["address"], "NA"),
            fdv_usd = safe_get(all_attributes, ["fdv_usd"], "NA"),
            market_cap_usd = safe_get(all_attributes, ["market_cap_usd"], "NA"),
            daily_volume = safe_get(all_attributes, ["volume_usd", "h24"], "NA"),
            daily_price_change = safe_get(all_attributes, ["price_change_percentage", "h24"], "NA"),
        )
        
        response_all.append(temp_dict)

    return response_all

In [ ]:
def get_new_pools(network, sort_by_col, max_pages=None):
    
    endpoint = f"/onchain/networks/{network}/new_pools"
    params = {}
    newpools_all = []
    page_count = 0

    # Follow pagination via the response links.next and collect across pages, with an optional max_pages cap.
    while True:
        pools_list_response = get_response(endpoint, use_pro, params, PRO_URL)
        if not pools_list_response:
            break

        newpools_all.extend(collect_response(pools_list_response))
        page_count += 1

        if max_pages is not None and page_count >= max_pages:
            break

        links = pools_list_response.get("links", {})
        next_link = links.get("next") if isinstance(links, dict) else None
        if not next_link:
            break

        parsed = urlparse(next_link)
        endpoint = parsed.path
        params = {k: v[0] for k, v in parse_qs(parsed.query).items()}

    df_new_pools = pd.DataFrame(newpools_all)

    # Change to local timezone
    df_new_pools["pool_created_at"] = pd.to_datetime(df_new_pools["pool_created_at"], utc=True)
    df_new_pools["pool_created_at"] = df_new_pools["pool_created_at"].dt.tz_convert("Europe/Berlin")

    return df_new_pools[df_new_pools["dex"] == "pump-fun"].sort_values(
        by=[f"{sort_by_col}"], ascending=False
    )


In [100]:
#get_new_pools("solana", "pool_created_at", max_pages = 5).head(50)

### Filter profitable pools

In [136]:
def collect_pool_response(list_response):

    response = list_response.get("data", {})
    all_attributes = response.get("attributes", {})
    daily_tx = all_attributes["transactions"]["h24"]
    rel = response["relationships"]
    
    # Safely extract launchpad_details or default to empty dict
    launchpad_details = all_attributes.get("launchpad_details", {})
        
    response_dict = dict(
        pair = all_attributes["name"],
        dex = rel["dex"]["data"]["id"],
        token_add = rel["base_token"]["data"]["id"].split("_")[1],
        pool_add = all_attributes["address"],
        pool_created_at = all_attributes["pool_created_at"],
        fdv_usd = all_attributes["fdv_usd"],
        market_cap_usd = all_attributes["market_cap_usd"],
        daily_volume = all_attributes["volume_usd"]["h24"],
        daily_price_change = all_attributes["price_change_percentage"]["h24"],
        daily_buys = daily_tx["buys"],
        daily_sells = daily_tx["sells"],
        daily_buyers = daily_tx["buyers"],
        daily_sellers = daily_tx["sellers"],
        grad_pert = (
            launchpad_details.get("graduation_percentage")
            if launchpad_details else 0
        ),
        completed = launchpad_details.get("completed", False),
        completed_at = launchpad_details.get("completed_at", None),
        dest_pool = launchpad_details.get("migrated_destination_pool_address", None)
    )

    return response_dict

In [137]:
def get_pool_data(network, pool_address):

    target_url = f"/onchain/networks/{network}/pools/{pool_address}"

    pool_list_response = get_response(target_url,
                                      use_pro,
                                      "",
                                      PRO_URL)

    pool_all = collect_pool_response(pool_list_response)

    return pool_all

In [138]:
def collect_pool_data(network, num_rows, max_pages):

    df_new_pools = get_new_pools(network, "pool_created_at", max_pages).head(num_rows)
    
    all_pool_data = []

    for pool_add in df_new_pools["pool_add"]:
        pool_data = get_pool_data(network, pool_add)
        all_pool_data.append(pool_data)

    df = pd.DataFrame(all_pool_data)

    df = df.astype({
        "pair": "string",
        "dex": "string",
        "pool_add": "string",
        "token_add": "string",
        "daily_buys": "Int64",
        "daily_sells": "Int64",
        "daily_buyers": "Int64",
        "daily_sellers": "Int64",
        "completed": "boolean",
        "dest_pool": "string",
    })

    # Numeric columns (coerce invalids to NaN)
    for col in ["fdv_usd", "market_cap_usd", "daily_volume", "daily_price_change", "grad_pert"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Timestamps
    df["pool_created_at"] = pd.to_datetime(df["pool_created_at"], utc=True, errors="coerce")
    df["completed_at"] = pd.to_datetime(df["completed_at"], utc=True, errors="coerce")

    return df

In [139]:
def analyze_pools(network, num_rows, max_pages=5):

    df_pool_data = collect_pool_data(network, num_rows, max_pages)

    # Inspect key metrics such as Fully Diluted Volume (FDV) and age of the pool. We want
    # to filter out pools which are older than 10 minutes and have FDV less than $5000.
    cutoff = pd.Timestamp.now(tz="UTC") - pd.Timedelta(minutes=10)

    df_filtered = df_pool_data[
                        (df_pool_data["pool_created_at"] >= cutoff) &
                        (df_pool_data["fdv_usd"] > 4000)
                        ].copy()

    # Convert to local timezone
    df_filtered["pool_created_at"] = df_filtered["pool_created_at"].dt.tz_convert("Europe/Berlin")
    
    return df_filtered

In [185]:
analyze_pools("solana", num_rows=200)

,pair,dex,token_add,pool_add,pool_created_at,fdv_usd,market_cap_usd,daily_volume,daily_price_change,daily_buys,daily_sells,daily_buyers,daily_sellers,grad_pert,completed,completed_at,dest_pool
3,PT / SOL,pump-fun,F1QTbhjhPB8Vm5FYKkG1sW8uTdWi8vuxRoPUZ5FYpump,EYMKQRVpD8Y6THHBxGedKeApYXmswkoXd8fmW1aFuz3q,2026-01-25 00:26:58+01:00,10621.152630,NaN,9744.513831,28.64,65,45,65,30,56.89,False,NaT,<NA>
6,OS / SOL,pump-fun,CBeDcTWE3ue2jRRrZNSDRgjfPG7wiKvdTVg6Hf3opump,CxKZ3WSdPmwHmQabry2Z15Xz2DWDsbbVTJw3PuhQac8e,2026-01-25 00:26:52+01:00,16361.956608,NaN,9035.282817,129.50,48,37,40,22,NaN,False,NaT,<NA>
10,PINGU / SOL,pump-fun,42xgXm1rBcX5m6n72FTeEbfg39FTtME12fJDnUEepump,9RLKfhngFsV8XWATDspFFwgnG7ER5X7SZWSiNAMKHW2j,2026-01-25 00:26:36+01:00,15694.755580,NaN,11146.844065,251.60,52,20,51,19,71.36,False,NaT,<NA>
11,BEGO / SOL,pump-fun,Gys72FmR3onY1ERXtQMf4zpWX9av632ra8y4Y3C1pump,6KEEVHnCrbHg2JKqa6pTL1vA4V5Ja7oWDooNekyE4tQ7,2026-01-25 00:26:29+01:00,4016.588456,NaN,127.125488,-44.02,11,9,2,1,NaN,False,NaT,<NA>


### Monitor real-time price

In [ ]:
WS_URL = f"wss://stream.coingecko.com/v1?x_cg_pro_api_key={CG_ANALYST_API_KEY}"
NETWORK_ID = "solana"

#### Stream and write to DuckDB database

In [186]:
DB_PATH = str(Path.cwd() / "price_stream_1.duckdb")

# One connection for writes
dbw = duckdb.connect(DB_PATH)

# Create new table
dbw.execute("""
CREATE TABLE IF NOT EXISTS price_stream (
    channel_type TEXT,
    network_id TEXT,
    token_address TEXT,
    usd_price DOUBLE,
    usd_price_24h_change_percentage DOUBLE,
    usd_market_cap DOUBLE,
    usd_24h_vol DOUBLE,
    last_updated_at TIMESTAMPTZ
);
""")

rename_map = {
    "c": "channel_type",
    "n": "network_id",
    "ta": "token_address",
    "p": "usd_price",
    "pp": "usd_price_24h_change_percentage",
    "m": "usd_market_cap",
    "v": "usd_24h_vol",
    "t": "last_updated_at",
}

async def stream_token_price_and_write_to_db(TOKEN_ADDRESS):

    async with websockets.connect(WS_URL) as ws:
        # 1) Subscribe
        subscribe_msg = {
            "command": "subscribe",
            "identifier": json.dumps({"channel": "OnchainSimpleTokenPrice"})
        }
        await ws.send(json.dumps(subscribe_msg))

        # 2) Send message to set tokens
        data_payload = {
            "network_id:token_addresses": [f"{NETWORK_ID}:{TOKEN_ADDRESS}"],
            "action": "set_tokens"
        }
        message_msg = {
            "command": "message",
            "identifier": json.dumps({"channel": "OnchainSimpleTokenPrice"}),
            "data": json.dumps(data_payload)
        }
        await ws.send(json.dumps(message_msg))

        # 3) Stream and then write data
        while True:
            msg = await ws.recv()
            payload = json.loads(msg)

            # Unwrap if needed
            if isinstance(payload, dict) and "message" in payload:
                data = payload["message"]
            else:
                data = payload

            # Only process if we get valid data
            if isinstance(data, dict) and "c" in data: 

                row = {rename_map[k]: payload.get(k) for k in rename_map}

                # Convert UNIX seconds to CET/CEST
                row["last_updated_at"] = (
                    pd.to_datetime(row["last_updated_at"], unit="s", utc=True)
                    .tz_convert("Europe/Berlin")
                )

                dbw.execute(
                    "INSERT INTO price_stream VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
                    [
                        row["channel_type"],
                        row["network_id"],
                        row["token_address"],
                        row["usd_price"],
                        row["usd_price_24h_change_percentage"],
                        row["usd_market_cap"],
                        row["usd_24h_vol"],
                        row["last_updated_at"],
                    ],
                )

#### Start background writer

In [ ]:
# To be run after we connect + subscribe and have `ws`
task = asyncio.create_task(stream_token_price_and_write_to_db("42xgXm1rBcX5m6n72FTeEbfg39FTtME12fJDnUEepump"))


In [194]:
#task.cancel()
#task.done()
#task.cancelled()

[t for t in asyncio.all_tasks() if not t.done()]


[<Task pending name='Task-135' coro=<WebSocketCommonProtocol.transfer_data() running at /home/vikas/.local/lib/python3.10/site-packages/websockets/legacy/protocol.py:955> wait_for=<Future pending cb=[Task.task_wakeup()]> cb=[Task.task_wakeup(), _wait.<locals>._on_completion() at /usr/lib/python3.10/asyncio/tasks.py:475]>,
 <Task pending name='Task-137' coro=<WebSocketCommonProtocol.close_connection() running at /home/vikas/.local/lib/python3.10/site-packages/websockets/legacy/protocol.py:1289> wait_for=<Task pending name='Task-135' coro=<WebSocketCommonProtocol.transfer_data() running at /home/vikas/.local/lib/python3.10/site-packages/websockets/legacy/protocol.py:955> wait_for=<Future pending cb=[Task.task_wakeup()]> cb=[Task.task_wakeup(), _wait.<locals>._on_completion() at /usr/lib/python3.10/asyncio/tasks.py:475]>>,
 <Task pending name='Task-134' coro=<stream_token_price_and_write_to_db() running at /tmp/ipykernel_523337/532832743.py:55> wait_for=<Future pending cb=[Task.task_wakeu

#### Read from database

In [195]:
dbr = duckdb.connect(DB_PATH)

df_latest = dbr.execute("""
    SELECT * FROM price_stream
    ORDER BY last_updated_at DESC
    LIMIT 10
""").df()

df_latest

,channel_type,network_id,token_address,usd_price,usd_price_24h_change_percentage,usd_market_cap,usd_24h_vol,last_updated_at
0,G1,solana,42xgXm1rBcX5m6n72FTeEbfg39FTtME12fJDnUEepump,0.00002,343.662428,19804.398366,0.0,2026-01-25 00:28:00+01:00
